KNN CLASSIFICATION

Traitement des données du fichier pickle

In [22]:
import pandas as pd

# Load the pickle file into a dataframe
df = pd.read_pickle('data/all_flow_data.pkl')

# Split the dataframe into two based on the "protocolName" column
df_HTTPWeb = df[df['appName'] == 'HTTPWeb']
df_SSH = df[df['appName'] == 'SSH']

# # Print the shapes of the resulting dataframes
print(f'Shape of HTTPWeb dataframe: {df_HTTPWeb.shape}')
print(f'Shape of SSH dataframe: {df_SSH.shape}')

# print(df.head())

Shape of HTTPWeb dataframe: (547347, 50)
Shape of SSH dataframe: (9697, 50)


Visualisation des données

In [24]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
df_HTTPWeb.head()

,appName,protocolName,origin,direction_L2L,direction_L2R,direction_R2L,direction_R2R,sourceCategory_MulticastNetwork,sourceCategory_PrivateNetwork,sourceCategory_PublicNetwork,destinationCategory_MulticastNetwork,destinationCategory_PrivateNetwork,destinationCategory_PublicNetwork,destinationCategory_UnknownNetwork,sourcePortCategory_WellKnownPorts,sourcePortCategory_RegisteredPorts,sourcePortCategory_DynamicPrivatePorts,destinationPortCategory_WellKnownPorts,destinationPortCategory_RegisteredPorts,destinationPortCategory_DynamicPrivatePorts,totalSourceBytesCategory_Small,totalSourceBytesCategory_Medium,totalSourceBytesCategory_Large,totalDestinationBytesCategory_Small,totalDestinationBytesCategory_Medium,totalDestinationBytesCategory_Large,totalSourcePacketsCategory_Low,totalSourcePacketsCategory_Medium,totalSourcePacketsCategory_High,totalDestinationPacketsCategory_Low,totalDestinationPacketsCategory_Medium,totalDestinationPacketsCategory_High,durationCategory_Short,tag_Attack,sourceTCPFlag_F,destinationTCPFlag_F,sourceTCPFlag_S,destinationTCPFlag_S,sourceTCPFlag_R,destinationTCPFlag_R,sourceTCPFlag_P,destinationTCPFlag_P,sourceTCPFlag_A,destinationTCPFlag_A,sourceTCPFlag_N/A,destinationTCPFlag_N/A,durationCategory_Medium,sensorInterfaceId,durationCategory_Long,startTime
1,HTTPWeb,tcp_ip,TestbedMonJun14Flows.xml,False,True,False,False,True,False,False,False,True,False,False,False,True,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False,NaN,NaN,NaN,NaN
2,HTTPWeb,tcp_ip,TestbedMonJun14Flows.xml,False,True,False,False,True,False,False,False,True,False,False,False,True,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False,NaN,NaN,NaN,NaN
3,HTTPWeb,tcp_ip,TestbedMonJun14Flows.xml,False,True,False,False,True,False,False,False,True,False,False,False,True,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False,NaN,NaN,NaN,NaN
9,HTTPWeb,tcp_ip,TestbedMonJun14Flows.xml,False,True,False,False,True,False,False,False,False,True,False,False,True,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,True,True,True,True,False,False,True,True,True,True,False,False,NaN,NaN,NaN,NaN
10,HTTPWeb,tcp_ip,TestbedMonJun14Flows.xml,False,True,False,False,True,False,False,False,False,True,False,False,True,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,True,True,True,True,False,False,True,True,True,True,False,False,NaN,NaN,NaN,NaN


In [23]:
# Check for NaN values in the DataFrame
nan_counts = df_SSH.isna().sum()
print("Number of NaN values in each column:")
print(nan_counts)

Number of NaN values in each column:
appName                                           0
protocolName                                      0
origin                                            0
direction_L2L                                     0
direction_L2R                                     0
direction_R2L                                     0
direction_R2R                                     0
sourceCategory_MulticastNetwork                   0
sourceCategory_PrivateNetwork                     0
sourceCategory_PublicNetwork                      0
destinationCategory_MulticastNetwork              0
destinationCategory_PrivateNetwork                0
destinationCategory_PublicNetwork                 0
destinationCategory_UnknownNetwork             1177
sourcePortCategory_WellKnownPorts                 0
sourcePortCategory_RegisteredPorts                0
sourcePortCategory_DynamicPrivatePorts            0
destinationPortCategory_WellKnownPorts            0
destinationPortCategory_Reg

Coupe les dataframes en 5 et effectue une boucle sur les 5 plis du dataset

In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import LabelEncoder

# Choix du modèle de classification
model = KNeighborsClassifier(n_neighbors=5)

# Initialisez l'objet KFold pour diviser les données en k plis
kf = KFold(n_splits=5)
i = 1

# Créez une liste pour stocker les métriques de performance pour chaque pli
performance_metrics = []

# Supprimer les colonnes à soucis
df_HTTPWeb = df_HTTPWeb.drop(columns='appName', axis=1)

# Bouclez sur les plis (S1, S2, S3, S4 et S5)
for train_index, test_index in kf.split(df_HTTPWeb):
    # Divisez les données en ensembles d'entraînement et de test pour ce pli
    x_train_set, x_test_set = df_HTTPWeb.iloc[train_index], df_HTTPWeb.iloc[test_index]

    # Récupération de la colonne avec l'étiquette à prédire
    y_train_set = x_train_set['tag_Attack']
    y_test_set = x_test_set['tag_Attack']

    # Preprocess Data (Standardisation des données numériques)
    # scaler = StandardScaler()
    # x_train_set_scaled = scaler.fit_transform(x_train_set)
    # x_test_set_scaled = scaler.transform(x_test_set)

    # Convert labels to numeric values
    le = LabelEncoder()
    y_train_set = le.fit_transform(y_train_set)
    y_test_set = le.transform(y_test_set)

    # Entraînez et évaluez votre modèle sur les ensembles d'entraînement et de test actuels
    model.fit(x_train_set, y_train_set)
    y_pred = model.predict(x_test_set)

    # Calcul des metrics
    accuracy = accuracy_score(y_test_set, y_pred)
    precision = precision_score(y_test_set, y_pred)
    recall = recall_score(y_test_set, y_pred)
    f1 = f1_score(y_test_set, y_pred)

    # Affichage des résultats
    print("=== Metrics for " + i + "split ===")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    i = i+1
    

# Calculez la moyenne des métriques de performance sur les k plis
# (par exemple, précision, rappel, F1-score, etc.)
average_performance = sum(performance_metrics) / len(performance_metrics)
print("Moyenne des performances :", average_performance)


ValueError: could not convert string to float: 'tcp_ip'